## Importing Libraries 

In [212]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import rgb2hex


# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import LabelEncoder


#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from folium import plugins

import urllib.request

from bs4 import BeautifulSoup

import matplotlib.pyplot as plt 

import colorsys

print('Libraries imported.')

Libraries imported.


## Gathering Data from Foursquare

In [61]:
CLIENT_ID = 'TDXC0KIMBCX5PNMWMVCHC0WRVVC2WUPF5KSZLFEHUPPMYS3W' # your Foursquare ID
CLIENT_SECRET = 'BYUUG12WKRFJUSV4DN3KZCTKF2QHAT1SP1WTLYHYGNFS1WYV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TDXC0KIMBCX5PNMWMVCHC0WRVVC2WUPF5KSZLFEHUPPMYS3W
CLIENT_SECRET:BYUUG12WKRFJUSV4DN3KZCTKF2QHAT1SP1WTLYHYGNFS1WYV


# Testing Sangenjaya Area

In [395]:
latitude = 35.6413
longitude = 139.6675
radius = 1000
queries = ['Italian', 'Coffee']
LIMIT = 150

In [396]:
def getNearbyVenues(queries):
    
    venues_list=[]
    for q in queries:
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT,
            q)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            q) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Category']
    
    return(nearby_venues)

In [472]:
nearby_venues = getNearbyVenues(queries)

In [473]:
nearby_venues.head()

,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Category
0,LOVEL DINING (ラブルダイニング),35.642739,139.667546,Italian Restaurant,Italian
1,caZé uzumaki,35.642771,139.668341,Italian Restaurant,Italian
2,ヨーロッパ食堂,35.645003,139.669534,Italian Restaurant,Italian
3,il Pizzaiolo (イル ピッツァイオーロ),35.644332,139.668702,Italian Restaurant,Italian
4,comodo kitchen,35.642195,139.672997,Italian Restaurant,Italian


In [474]:
labelencoder = LabelEncoder()
nearby_venues['Category Type'] = labelencoder.fit_transform(nearby_venues['Category'])
nearby_venues.head(10)

,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Category,Category Type
0,LOVEL DINING (ラブルダイニング),35.642739,139.667546,Italian Restaurant,Italian,1
1,caZé uzumaki,35.642771,139.668341,Italian Restaurant,Italian,1
2,ヨーロッパ食堂,35.645003,139.669534,Italian Restaurant,Italian,1
3,il Pizzaiolo (イル ピッツァイオーロ),35.644332,139.668702,Italian Restaurant,Italian,1
4,comodo kitchen,35.642195,139.672997,Italian Restaurant,Italian,1
5,Rainbow,35.644361,139.670262,Italian Restaurant,Italian,1
6,Vineria il Chiasso,35.640591,139.674575,Italian Restaurant,Italian,1
7,ワイン食堂 INASEYA MARE,35.643214,139.662873,Italian Restaurant,Italian,1
8,Bricca,35.640972,139.673939,Italian Restaurant,Italian,1
9,Trattoria e Pizzeria L'ARTE,35.643074,139.672357,Italian Restaurant,Italian,1


## Plotting restaurants on map

In [475]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(2)
ys = [i+x+(i*x)**2 for i in range(2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, name, cat, col in zip(nearby_venues['Venue Latitude'], nearby_venues['Venue Longitude'], nearby_venues['Venue Name'], nearby_venues['Category'], nearby_venues['Category Type']):
    label = folium.Popup(str(cat) + '\n'+ str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[col-1],
        fill=True,
        fill_color=rainbow[col-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [476]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

clusters = plugins.MarkerCluster().add_to(map_clusters)

x = np.arange(2)
ys = [i+x+(i*x)**2 for i in range(2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, name, cat, col in zip(nearby_venues['Venue Latitude'], nearby_venues['Venue Longitude'], nearby_venues['Venue Name'], nearby_venues['Category'], nearby_venues['Category Type']):
    label = folium.Popup(str(cat) + '\n'+ str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[col-1],
        fill=True,
        fill_color=rainbow[col-1],
        fill_opacity=0.7).add_to(clusters)
       
map_clusters

## K-means on Coffee and Italian food seperately 

### Creating the input data

In [490]:
Coffee = nearby_venues.loc[nearby_venues['Category Type'] == 0] 
Italian = nearby_venues.loc[nearby_venues['Category Type'] == 1]
Coffee = Coffee[['Venue Latitude','Venue Longitude']]
Italian = Italian[['Venue Latitude','Venue Longitude']]

### k-means on Coffee

In [491]:
k_means = KMeans(init = "k-means++", n_clusters = 10, n_init=12)
k_means.fit(Coffee)
k_means_labels = k_means.labels_
Coffee['Label'] = k_means_labels

### k-means on Italian

In [492]:
k_means = KMeans(init = "k-means++", n_clusters = 10, n_init=12)
k_means.fit(Italian)
k_means_labels = k_means.labels_
Italian['Label'] = k_means_labels

### Creating Zones

In [493]:
Coffee_Zones = []

Coffee_Zones = Coffee.groupby('Label').count()
Coffee_Zones = Coffee_Zones.drop('Venue Longitude',1)
Coffee_Zones.columns=['Count']
Coffee_Zones = pd.concat([Coffee.groupby('Label').mean(), Coffee_Zones], axis = 1) 

In [494]:
Italian_Zones = []

Italian_Zones = Italian.groupby('Label').count()
Italian_Zones = Italian_Zones.drop('Venue Longitude',1)
Italian_Zones.columns=['Count']
Italian_Zones = pd.concat([Italian.groupby('Label').mean(), Italian_Zones], axis = 1) 

In [495]:
Italian_Zones.sort_values('Count', ascending = False)

,Venue Latitude,Venue Longitude,Count
Label,,,
9,35.644661,139.669929,11
7,35.642637,139.668303,10
1,35.642774,139.671979,9
5,35.641158,139.674108,5
3,35.638236,139.667478,4
2,35.638837,139.673238,3
8,35.642781,139.661268,3
0,35.643239,139.663591,2
4,35.647326,139.669094,2


In [496]:
Coffee_Zones.sort_values('Count', ascending = False)

,Venue Latitude,Venue Longitude,Count
Label,,,
1,35.643287,139.669788,20
7,35.643146,139.671923,7
0,35.640021,139.674534,5
3,35.640385,139.670994,5
8,35.646304,139.670547,4
2,35.646256,139.660810,3
4,35.649213,139.667968,3
6,35.644106,139.665658,3
5,35.635465,139.662173,1


## Plotting Zones

In [500]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)


# add Coffee markers to the map
for lat, lng, count in zip(Coffee_Zones['Venue Latitude'], Coffee_Zones['Venue Longitude'], Coffee_Zones['Count']):
    label = folium.Popup('Coffee ' + str(count) + ' Locations', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=50,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=count/22).add_to(map_clusters)
    
# add Italian markers to the map
for lat, lng, count in zip(Italian_Zones['Venue Latitude'], Italian_Zones['Venue Longitude'], Italian_Zones['Count']):
    label = folium.Popup('Italian Location-' + str(count) , parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=50,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=count/22).add_to(map_clusters)
       
map_clusters

In [523]:
Italian[Italian['Label'].isin([3,4])]

,Venue Latitude,Venue Longitude,Label
22,35.638462,139.667464,3
23,35.638473,139.668007,3
25,35.638218,139.668060,3
29,35.637790,139.666380,3
43,35.646284,139.669689,4
49,35.648367,139.668500,4


In [529]:
nearby_venues.iloc[[22,23,25,29,43,49],:]

,Venue Name,Venue Latitude,Venue Longitude,Venue Category,Category,Category Type
22,Euro,35.638462,139.667464,Italian Restaurant,Italian,1
23,HAMANO,35.638473,139.668007,Italian Restaurant,Italian,1
25,ナティーボ NATIVO,35.638218,139.668060,Italian Restaurant,Italian,1
29,Osteria Due (オステリア ドゥエ),35.637790,139.666380,Italian Restaurant,Italian,1
43,3,35.646284,139.669689,Italian Restaurant,Italian,1
49,taverna ichi,35.648367,139.668500,Italian Restaurant,Italian,1
